偏差与方差：利用水库水位变化预测大坝出水量

In [ ]:
import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt   
import scipy.io as sio  
from scipy.optimize import minimize
from sklearn.preprocessing import PolynomialFeatures
data=sio.loadmat(r"C:\Users\HP\Desktop\高速下载-17457248894892024吴恩达资料\2014版 吴恩达机器学习全套资料\吴恩达机器学习课后作业源码及笔记\ex5-bias vs variance\ex5data1.mat")

In [ ]:
#训练集
x=data['X']
y=data['y']
#验证集
xv=data['Xval']
yv=data['yval']
#测试集
xt=data['Xtest']
yt=data['ytest']
fig,ax=plt.subplots(3,1,figsize=(8,12))
ax[0].scatter(x,y)
ax[1].scatter(xv,yv)
ax[2].scatter(xt,yt)

In [ ]:
#训练训练集
def pf(x):
    return (x-x.mean())/x.std()#当多项式映射时纬度高必须要进行特征归一化，不然会梯度爆炸
poly=PolynomialFeatures(degree=6)
x=pf(x)
x_po=poly.fit_transform(x)
xv=pf(xv)#注意必须要特征归一化
xv_po=poly.transform(xv)
xt=pf(xt)
xt_po=poly.transform(xt)
def costFunction(x,y,theta,la):
    inner=np.power(x@theta-y,2)
    ou=np.sum(inner)/2/len(x)
    re=la/2/len(x)*np.sum(theta[1:]**2)
    return re+ou
def tidu(x,y,theta,a,c,la,xv,yv):#训练集与测试集的损失函数比较
    costs=[]
    cv_cost=[]
    for i in range(c):
        re=la/len(x)*theta
        re[0]=0
        theta=theta-(x.T@(x@theta-y)/len(x)+re)*a
        costs.append(costFunction(x,y,theta,la))
        cv_cost.append(costFunction(xv,yv,theta,la))
    return theta,costs,cv_cost
theta=np.zeros((x_po.shape[1],1))
a=0.01
c=3000
la=2
theta,costs,cv_cost=tidu(x_po,y,theta,a,c,la,xv_po,yv)
fig,ax=plt.subplots()
ax.plot(np.arange(c),costs)
ax.plot(np.arange(c),cv_cost)
plt.show()
x1=np.linspace(-1.5,2,100)
x1_po=x1.reshape(len(x1),1)
x1_po=poly.transform(x1_po)
x1=x1.reshape(len(x1),1)

In [ ]:
#训练集观察
fig,ax=plt.subplots()
ax.scatter(x,y)
ax.plot(x1,x1_po@theta,label='train')
ax.legend()
plt.show()
#测试集观察
fig,ax=plt.subplots()
ax.scatter(xv,yv)
ax.plot(x1,x1_po@theta,label='val')
ax.legend()
plt.show()
#验证集观察
fig,ax=plt.subplots()
ax.scatter(xt,yt)
ax.plot(x1,x1_po@theta,label='tcv')
ax.legend()
plt.show()